In [1]:
from abc import ABC

from gulpio2 import GulpDirectory
from pathlib import Path
from moviepy.editor import ImageSequenceClip, clips_array

from collections import defaultdict

import pickle
import pandas as pd
from pathlib import Path

import torch as t
import torch.nn as nn
import numpy as np
from omegaconf import OmegaConf
from typing import Any, Dict, List, Sequence, Union

from systems import EpicActionRecognitionSystem
from systems import EpicActionRecogintionDataModule

from utils.metrics import compute_metrics
from utils.actions import action_id_from_verb_noun
from scipy.special import softmax

from GPUtil import showUtilization as gpu_usage
from tqdm import tqdm

In [2]:
verb_noun_link = {}
verb_class_link = {}

verbs = pd.read_csv('../datasets/epic/labels/EPIC_100_verb_classes.csv')
nouns = pd.read_csv('../datasets/epic/labels/EPIC_100_noun_classes.csv')


rgb_train = GulpDirectory('../datasets/epic/gulp/rgb/p01_rgb_test/')

_, meta = rgb_train['P01_01_96']

# verbs['key'][meta['verb']]
meta
# verbs['key'][meta['verb_class']]

# for i, c in enumerate(tqdm(rgb_train)):
#     if i == 0:
#         for _, meta in c:
#             if meta['verb'] in verb_noun_link:
#                 verb_noun_link[meta['verb']].append(meta['noun'])
#             else:
#                 verb_noun_link[meta['verb']] = [meta['noun']] 
#             if meta['verb_class'] in verb_class_link:
#                 verb_class_link[meta['verb_class']].append(meta['noun_class'])
#             else:
#                 verb_class_link[meta['verb_class']] = [meta['noun_class']]

{'narration_id': 'P01_01_96',
 'participant_id': 'P01',
 'video_id': 'P01_01',
 'narration_timestamp': '00:07:46.520',
 'start_timestamp': '00:07:49.14',
 'stop_timestamp': '00:07:50.32',
 'start_frame': 28148,
 'stop_frame': 28219,
 'narration': 'take glass',
 'verb': 'take',
 'verb_class': 0,
 'noun': 'glass',
 'noun_class': 10,
 'all_nouns': ['glass'],
 'all_noun_classes': [10],
 'frame_size': [256, 456, 3],
 'num_frames': 72}

In [35]:
seen = set()
verb_noun_unique = [x for x in verb_noun_link['open'] if not (x in seen or seen.add(x))]
verb_noun_link['open'], verb_noun_unique

# verb_class_link[3]

# xd = list(set(verb_noun_link['take']))

# with open('../datasets/epic/labels/verb_noun.pkl', 'wb') as f:
#     pickle.dump(verb_noun_link, f)

# with open('../datasets/epic/labels/verb_noun_unique.pkl', 'wb') as f:
#     pickle.dump({verb: list(set(verb_noun_link[verb])) for verb in verb_noun_link.keys()} , f)


(['door',
  'drawer',
  'cupboard',
  'container',
  'drawer',
  'drawer',
  'fridge',
  'cupboard',
  'cupboard',
  'salt',
  'drawer',
  'drawer',
  'cupboard',
  'rice'],
 ['door', 'drawer', 'cupboard', 'container', 'fridge', 'salt', 'rice'])

In [3]:
with open('../datasets/epic/labels/verb_noun_unique.pkl', 'rb') as pkl:
    verb_noun = pickle.load(pkl)
    
with open('../datasets/epic/labels/verb_noun_classes_unique.pkl', 'rb') as pkl:
    verb_noun_classes = pickle.load(pkl)

In [8]:
for k,j in zip(verb_noun.keys(), verb_noun_classes.keys()):
    assert len(verb_noun[k]) == len(verb_noun_classes[j])

In [24]:
v = 5

def callback(verb_class):
    
    noun_list=verb_noun[list(verb_noun.keys())[verb_class]]
    noun_class=verb_noun_classes[list(verb_noun_classes.keys())[verb_class]]
    
    assert len(noun_list) == len(noun_class)
    
    return [{'label':i, 'value':j} for i,j in zip(noun_list,noun_class)]

In [25]:
list(verb_.keys())[v], callback(v)

('close',
 [{'label': 'box', 'value': 3},
  {'label': 'drawer', 'value': 6},
  {'label': 'rosemary', 'value': 8},
  {'label': 'freezer', 'value': 138},
  {'label': 'jar', 'value': 12},
  {'label': 'oregano', 'value': 15},
  {'label': 'lid', 'value': 145},
  {'label': 'powder', 'value': 19},
  {'label': 'cupboard', 'value': 21},
  {'label': 'bag', 'value': 22},
  {'label': 'olive', 'value': 23},
  {'label': 'cooker:slow', 'value': 152},
  {'label': 'top', 'value': 277},
  {'label': 'oven', 'value': 31},
  {'label': 'milk', 'value': 36},
  {'label': 'fridge', 'value': 38},
  {'label': 'bin', 'value': 40},
  {'label': 'bottle', 'value': 42},
  {'label': 'oil', 'value': 44},
  {'label': 'salt', 'value': 46},
  {'label': 'liquid:washing', 'value': 64},
  {'label': 'kettle', 'value': 83},
  {'label': 'container', 'value': 113}])

In [12]:
with open('../datasets/epic/labels/verb_noun_unique.pkl', 'rb') as pkl:
    vn = pickle.load(pkl)

with open('../datasets/epic/labels/verb_noun_classes_unique.pkl', 'rb') as pkl:
    vnn = pickle.load(pkl)

In [26]:
np.array((vn['take'], vnn[0])).T


array([['box', '1'],
       ['dough', '2'],
       ['sauce', '3'],
       ['beer', '4'],
       ['yeast', '5'],
       ['candle', '6'],
       ['glass', '7'],
       ['powder', '8'],
       ['spatula', '9'],
       ['tray', '10'],
       ['onion', '13'],
       ['rice', '14'],
       ['lighter', '15'],
       ['olive', '16'],
       ['pepper', '17'],
       ['milk', '18'],
       ['lettuce', '19'],
       ['fruit', '20'],
       ['egg', '21'],
       ['salt', '22'],
       ['courgette', '23'],
       ['tongs', '24'],
       ['celery', '25'],
       ['choi:pak', '26'],
       ['maker:coffee', '29'],
       ['aubergine', '30'],
       ['towel', '31'],
       ['cup', '32'],
       ['cutlery', '33'],
       ['knife', '34'],
       ['noodle', '35'],
       ['clip', '36'],
       ['plate', '37'],
       ['clothes', '38'],
       ['bag', '39'],
       ['bread', '41'],
       ['leaf', '43'],
       ['scissors', '44'],
       ['floor', '47'],
       ['bowl', '48'],
       ['stock', '49'],
     

In [4]:
with open('../datasets/epic/features/p01_features.pkl','rb') as f:
    features = pickle.load(f)
    
features['narration_id'].index('P01_01_289')

1210

In [5]:
features['labels'][1210]

{'narration_id': 'P01_01_289',
 'participant_id': 'P01',
 'video_id': 'P01_01',
 'narration_timestamp': '00:23:56.100',
 'start_timestamp': '00:23:57.14',
 'stop_timestamp': '00:23:57.82',
 'start_frame': 86228,
 'stop_frame': 86269,
 'narration': 'take curry powder',
 'verb': 'take',
 'verb_class': 0,
 'noun': 'powder:curry',
 'noun_class': 138,
 'all_nouns': ['powder:curry'],
 'all_noun_classes': [138],
 'frame_size': [256, 456, 3],
 'num_frames': 42}